# DFS Benchmark Injection Testing

In [2]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
%run "../../Setup_Scripts/Setup_Generic.ipynb"
%matplotlib widget
scope.adc.samples = 800

INFO: Found ChipWhisperer😍


In [ ]:
scope.dis()
target.dis()

In [3]:
#import necessary library 
import linear_comp as lc
import matplotlib.pyplot as plt 
import chipwhisperer as cw
import numpy as np

In [4]:
def cap_pass_trace():
    scope.arm()
    ret = scope.capture()
    num_char = target.in_waiting()
    while num_char > 0:
        target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()

    trace = scope.get_last_trace()
    return trace

<h3> This is for Research Benchmark Checking </h3>

In [ ]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../hardware/victims/firmware/DFS
gmake PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

In [13]:
cw.program_target(scope, prog, "../../hardware/victims/firmware/DFS/dfs-{}.hex".format(PLATFORM))

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2061 bytes


In [7]:
#this is the method bloc get all the possible traits from the testing, repeatedly. 
reset_target(scope)
trace_set=[]
for i in range(25):
    trace_hit = cap_pass_trace()
    if(len(trace_set))==0:
        trace_set.append(trace_hit)
    else:
        flag=False
        for j in range(len(trace_set)):
            if(lc.linear_comp(trace_hit,trace_set[j],0.1,50))>750:
                flag=True
        
        if not flag:
            trace_set.append(trace_hit)
    
print(len(trace_set))

2


In [ ]:
cw.plot(trace_set[0])*cw.plot(trace_set[1])

In [ ]:
#reset_target(scope)
#trace_hit = cap_pass_trace()
#cw.plot(trace_hit) *cw.plot(trace_set[0]) 
#lc.linear_comp(trace_hit,trace_set[0],0.1,50)

# The Testing Part

In [14]:
result=[]
reset_target(scope)

for i in range(1024): # we now are getting right result
    trace_test=cap_pass_trace()
    x=max([lc.linear_comp(trace_test,trace_set[j],0.1,250) for j in range(len(trace_set))])
    result.append(x)

In [11]:
with open("DFS_Accuracy.csv",'w') as f:
    f.write("Test Iteration,Matching,Decision\n")
    for i in range(len(result)):
        if(result[i]>750):
            f.write(f"{i},{result[i]},OK\n")
        else:
            f.write(f"{i},{result[i]},Failed\n")